In [ ]:
# Valence bands offset calculation for GaAs/AlAs interface with Mat3ra workflows.py package
# This script is based on the following paper:
# "Accurate and efficient band-offset calculations from density functional theory" by L. Weston, ..., Van de Walle


In [ ]:
from mat3ra.workflows import BandOffsetWorkflowConfiguration
subworkflow_1_configuration = BandOffsetWorkflowConfiguration()
subworkflow_2_configuration = BandOffsetWorkflowConfiguration()
workflow_configuration = BandOffsetWorkflowConfiguration(k_points=[6,6,1], k_path=["G", "M", "K", "G"], subworkflow_configurations=[subworkflow_1_configuration, subworkflow_2_configuration])

In [ ]:
# Enable interactive selection of materials or use default configurations
IS_INTERACTIVE_SELECTION = False

# Indices to access materials from the loaded list
INTERFACE_INDEX = 0  # Index for the interface material
INTERFACE_LEFT_INDEX = 1  # Index for the left material
INTERFACE_RIGHT_INDEX = 2  # Index for the right material

# Workflow-specific parameters
MAX_AREA = 50  # Maximum area for strain matching in Angstrom^2

from utils.jupyterlite import get_materials

# Load materials; ensure materials are pre-selected in the outer runtime
materials = get_materials(globals())

try:
    interface_material = materials[INTERFACE_INDEX]
    interface_left_material = materials[INTERFACE_LEFT_INDEX]
    interface_right_material = materials[INTERFACE_RIGHT_INDEX]
except IndexError as e:
    print(f"Error accessing materials: {e}")
    # Fallback assignments if indices are out of range
    interface_material = materials[0]
    interface_left_material = materials[1] if len(materials) > 1 else materials[0]
    interface_right_material = materials[2] if len(materials) > 2 else materials[0]


2. Configure VBO Workflow
Set parameters specific to the VBO workflow, such as calculation methods, indices, and other configurations.

2.1. Define Unit Configurations


In [ ]:
from workflow_package import UnitConfiguration, SubworkflowConfiguration, WorkflowConfiguration


# Interface-related Units
unit_interface_set_index = UnitConfiguration(
    type="assignment",
    attributes={
        "name": "Set Material Index (Interface)",
        "value": "0"
    }
)

unit_interface_find_minima = UnitConfiguration(
    type="assignment",
    attributes={
        "operand": "AVG_ESP_INTERFACE"
    }
)

# Interface-left-related Units
unit_interface_left_set_index = UnitConfiguration(
    type="assignment",
    attributes={
        "name": "Set Material Index (Interface left)",
        "value": "1"
    }
)

unit_interface_left_execution_band_gap = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "pw-bands-calculate-band-gap-left"
    }
)

unit_interface_left_assign_band_gaps = UnitConfiguration(
    type="assignment",
    attributes={
        "input": [
            {"name": "band_gaps", "scope": "pw-bands-calculate-band-gap-left"}
        ]
    }
)

unit_interface_left_assign_vbm = UnitConfiguration(
    type="assignment",
    attributes={
        "operand": "VBM_LEFT"
    }
)

unit_interface_left_execution_avg_esp = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "average-electrostatic-potential-left"
    }
)

unit_interface_left_assign_avg_potential = UnitConfiguration(
    type="assignment",
    attributes={
        "input": [
            {"name": "average_potential_profile", "scope": "average-electrostatic-potential-left"}
        ]
    }
)

unit_interface_left_execution_find_extrema = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "python-find-extrema-left"
    }
)

unit_interface_left_find_minima = UnitConfiguration(
    type="assignment",
    attributes={
        "operand": "AVG_ESP_LEFT",
        "input": [
            {"name": "STDOUT", "scope": "python-find-extrema-left"}
        ]
    }
)

# Interface-right-related Units
unit_interface_right_set_index = UnitConfiguration(
    type="assignment",
    attributes={
        "name": "Set Material Index (Interface right)",
        "value": "2"
    }
)

unit_interface_right_execution_band_gap = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "pw-bands-calculate-band-gap-right"
    }
)

unit_interface_right_assign_band_gaps = UnitConfiguration(
    type="assignment",
    attributes={
        "input": [
            {"name": "band_gaps", "scope": "pw-bands-calculate-band-gap-right"}
        ]
    }
)

unit_interface_right_assign_vbm = UnitConfiguration(
    type="assignment",
    attributes={
        "operand": "VBM_RIGHT"
    }
)

unit_interface_right_execution_avg_esp = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "average-electrostatic-potential-right"
    }
)

unit_interface_right_assign_avg_potential = UnitConfiguration(
    type="assignment",
    attributes={
        "input": [
            {"name": "average_potential_profile", "scope": "average-electrostatic-potential-right"}
        ]
    }
)

unit_interface_right_execution_find_extrema = UnitConfiguration(
    type="executionBuilder",
    attributes={
        "flowchartId": "python-find-extrema-right"
    }
)

unit_interface_right_find_minima = UnitConfiguration(
    type="assignment",
    attributes={
        "operand": "AVG_ESP_RIGHT",
        "input": [
            {"name": "STDOUT", "scope": "python-find-extrema-right"}
        ]
    }
)


2.2.Define SubWorkflow Configurations

In [ ]:

# Interface-related SubWorkflows
subworkflow_interface_avg_esp = SubworkflowConfiguration(
    name="BS + Avg ESP (Interface)",
    unit_configs=[unit_interface_set_index]
)

subworkflow_interface_find_minima = SubworkflowConfiguration(
    name="Find ESP Values (Interface)",
    unit_configs=[unit_interface_find_minima]
)

# Interface-left-related SubWorkflows
subworkflow_interface_left_avg_esp = SubworkflowConfiguration(
    name="BS + Avg ESP (interface left)",
    unit_configs=[
        unit_interface_left_set_index,
        unit_interface_left_execution_band_gap,
        unit_interface_left_assign_band_gaps,
        unit_interface_left_assign_vbm,
        unit_interface_left_execution_avg_esp,
        unit_interface_left_assign_avg_potential
    ]
)

subworkflow_interface_left_find_minima = SubworkflowConfiguration(
    name="Find ESP Value (Interface left)",
    unit_configs=[
        unit_interface_left_execution_find_extrema,
        unit_interface_left_find_minima
    ]
)

# Interface-right-related SubWorkflows
subworkflow_interface_right_avg_esp = SubworkflowConfiguration(
    name="BS + Avg ESP (interface right)",
    unit_configs=[
        unit_interface_right_set_index,
        unit_interface_right_execution_band_gap,
        unit_interface_right_assign_band_gaps,
        unit_interface_right_assign_vbm,
        unit_interface_right_execution_avg_esp,
        unit_interface_right_assign_avg_potential
    ]
)

subworkflow_interface_right_find_minima = SubworkflowConfiguration(
    name="Find ESP Value (Interface right)",
    unit_configs=[
        unit_interface_right_execution_find_extrema,
        unit_interface_right_find_minima
    ]
)

# Final VBO Calculation SubWorkflow
subworkflow_final_vbo = SubworkflowConfiguration(
    name="Valence Band Offset Calculation",
    # ???
)


2.3. Assemble Main Workflow Configuration

In [ ]:


main_workflow = WorkflowConfiguration(
    name="Valence Band Offset (2D)",
    units=[
        # Interface-related SubWorkflows
        subworkflow_interface_avg_esp,
        subworkflow_interface_find_minima,
        
        # Interface-left-related SubWorkflows
        subworkflow_interface_left_avg_esp,
        subworkflow_interface_left_find_minima,
        
        # Interface-right-related SubWorkflows
        subworkflow_interface_right_avg_esp,
        subworkflow_interface_right_find_minima,
        
        # Final VBO Calculation SubWorkflow
        subworkflow_final_vbo
    ]
)

> Submit via API point function or convert to YAML?

4.2. Convert to YAML
Serialize the workflow into a YAML format for platform compatibility.

In [ ]:
import yaml

def serialize_unit(unit):
    if isinstance(unit, Workflow.SubWorkflow):
        return {
            "name": unit.name,
            "type": unit.type,
            "config": unit.config,
            "unitConfigs": [
                {
                    "index": uc.index,
                    "type": uc.type,
                    "config": uc.config
                } for uc in unit.unitConfigs
            ]
        }
    else:
        # Handle other unit types if necessary
        return {}

# Convert the workflow to a dictionary
workflow_dict = {
    "name": vbo_workflow.name,
    "units": [serialize_unit(u) for u in vbo_workflow.units]
}

# Serialize the dictionary to a YAML string
workflow_yaml = yaml.dump(workflow_dict, sort_keys=False)
print(workflow_yaml)


4.3. Send to Platform
Send the serialized YAML workflow to your platform. This step may vary depending on your platform's API or integration method.

In [ ]:
import requests

# Example: Send the YAML to a platform endpoint
platform_url = "https://your-platform-endpoint/api/workflows"  # Replace with actual URL
headers = {
    "Content-Type": "application/x-yaml",
    "Authorization": "Bearer YOUR_API_TOKEN"  # Replace with actual token if needed
}

response = requests.post(platform_url, data=workflow_yaml, headers=headers)

if response.status_code == 200:
    print("Workflow successfully sent to the platform.")
else:
    print(f"Failed to send workflow. Status code: {response.status_code}")
    print(f"Response: {response.text}")
